#### Imports

In [ ]:
import cv2
import numpy as np
import open3d as o3d
import os

In [ ]:
def save_point_cloud(file_path, point_cloud):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    # Save to PLY file
    o3d.io.write_point_cloud(file_path, pcd)

def read_depth_map(file_path):
    depth_image = cv2.imread(file_path) 
    B = depth_image[:, :, 0]
    G = depth_image[:, :, 1]
    R = depth_image[:, :, 2]
    normalized = (G + B * 256 + R * 256 * 256) / (256 * 256 - 1)
    depth_map = normalized * 1000
    return depth_map

def mask_depth_map(depth_map, mask_image):
    print(np.unique(mask_image, return_counts=True))
    depth_map[mask_image == 0] = 0
    return depth_map


def calculate_vertical_fov(image_width, image_height, fov_x):
    aspect_ratio = image_width / image_height
    fov_x_rad = np.radians(fov_x)
    fov_y_rad = 2 * np.arctan(np.tan(fov_x_rad / 2) / aspect_ratio)
    fov_y = np.degrees(fov_y_rad)
    return fov_y

def calculate_focal_length(image_width, fov_x):
    return image_width / (2 * np.tan(np.radians(fov_x) / 2))

def calculate_pixel_focal_length(image_width, image_height, fov_x, fov_y):
    """
    Calculate the focal length based on the field of view and image dimensions.
    
    Parameters:
    - fov_x: Horizontal field of view in degrees.
    - image_width: Width of the image in pixels.
    - image_height: Height of the image in pixels.
    
    Returns:
    - fx: Focal length in the x direction.
    - fy: Focal length in the y direction.
    - cx: Principal point x-coordinate.
    - cy: Principal point y-coordinate.
    """
    fov_x_rad = np.radians(fov_x)
    fov_y_rad = np.radians(fov_y)
    
    
    fx = image_width / (2 * np.tan(fov_x_rad / 2))
    fy = image_height / (2 * np.tan(fov_y_rad / 2))
    
    cx = image_width / 2.0
    cy = image_height / 2.0
    
    return fx, fy, cx, cy


def depth_map_to_point_cloud(depth_map, fov_x, fov_y):
    """
    Converts a depth image to a point cloud and stores it as a PLY file.
    
    Parameters:
    - depth_image: numpy array of shape (H, W) containing depth information.
    - image_width: Width of the image in pixels.
    - image_height: Height of the image in pixels.
    """
    # CARLA default horizontal FoV value

    height, width = depth_map.shape
    fx, fy, cx, cy = calculate_pixel_focal_length(width, height, fov_x, fov_y)


    point_cloud = []
    for v in range(height):
        for u in range(width):
            depth = depth_map[v, u]
            if depth <= 0 or depth > 25:  # Skip invalid depth
                continue
            x = (u - cx) / fx
            y = (v - cy) / fy
            xy_depth = np.sqrt(np.square(x) + np.square(y))
            z = np.sqrt(np.square(depth) - np.square(xy_depth))

            point_cloud.append([x, y, z])

    # Convert to Open3D point cloud
    point_cloud_np = np.array(point_cloud)
    return point_cloud_np

# Replace with your depth image and camera parameters
DATA_DIR = "generated_data"
depth_map = read_depth_map(f"{DATA_DIR}/DEPTH_CAM_FRONT/6784670033.png")
depth_map = mask_depth_map(depth_map, cv2.imread(f"{DATA_DIR}/SEMANTIC_CAM_FRONT/6784670033.mask.png", cv2.IMREAD_GRAYSCALE))
image_width, image_height = depth_map.shape[:2]
fov_x, fov_y = 120.0, 90 #calculate_vertical_fov(image_width, image_height, 120)
point_cloud = depth_map_to_point_cloud(depth_map, fov_x, fov_y)
save_point_cloud(f"{DATA_DIR}/BEV/point_cloud.ply", point_cloud)

In [ ]:
z_coordinates = point_cloud[:, 1]
print(np.min(z_coordinates), np.max(z_coordinates))
filtered_point_cloud = point_cloud[z_coordinates > 0.1]
save_point_cloud(f"{DATA_DIR}/BEV/filtered_point_cloud.ply", filtered_point_cloud)